# Addressing the Rare Word Problem in Neural Machine Translation
Minh-Thang Luong et al.

## 0. あぶすと
NMTは従来の手法より良い結果出してるすげーやつ  
でも従来の小規模の語彙でOOVの可能性がある単語をUNKタグで表現するようなend2endのNMTモデルだと、低頻度語句をちゃんと翻訳できない  
本論文ではこの問題に効率的に対処する手法を提案する  

ターゲット文に出現する各OOV単語について、NMTシステムがソース文の関連する単語の位置を出力することを可能とする単語アライメントアルゴリズムの出力によって補完されたデータを用いてNMTシステムを訓練する  
この(位置)情報は、各OOV単語を辞書を用いて翻訳する後処理のステップで利用される

## 1. いんとろ
NMTはナウでヤングでいけてるすごいの  
一方で従来のNMTは、低頻度語を翻訳できないみたいなところある  
なぜなら固定サイズの小規模語彙しか持ってないから、多くのOOV単語を表現するのに*unk*シンボル一つを使うしかない  
低頻度を多く含む文章は、頻出語を主として含む文章よりも貧弱な翻訳をされる傾向にある  
一方で通常のフレーズベースドなシステムでは、より多くの語彙を用いることができる上、明示的なアライメントやフレーズテーブルによって低頻度語の翻訳をメモ化できるため、同様の問題は起こらない  
このようなフレーズベースドなシステムの強みをモチベーションに、NMTにおける低頻度語の問題に対処する新しい手法を提案する  

提案手法では、各OOV単語についてそれと関係するソース文の単語を指す"ポインター"をNMTが出力できるような、明示的なアライメント情報を訓練コーパスにアノテートする  
この情報は後処理のステップで翻訳が見つからないOOV単語を辞書を使って翻訳する際に使われる  

## 2. NMT
いつもの  
LSTM使ったenc-decモデル使う  

NMTの先行研究で低頻度語問題に対処したものはない(2015年時点)けど、Jeanら(2015)がでかい語彙を扱うためにsoftmaxを効率的に近似する手法を最近提案した  
でも大きな語彙があっても、人名とか数字とかでは依然として問題が残る  
それに対処するためにはワイらの提案するみたいな手法を使ったほうがよいってjeanらは言ってて、彼らのアプローチと相補的であるといえる  

## 3. Rare Word Models
非常に多くのNMTの研究があるにも関わらず、Jeanらのそれを除けば低頻度語を対処する研究はまったくなかった  

で、こういう低頻度語に対処するために、ターゲット文における未知語の元を辿れるようなNMTの学習を提案する  
もしターゲット文の各未知語に対応するソース文の単語がわかってたら、システムが出力した*unk*を後処理で辞書なりなんなり使って翻訳すればいい  
このようなアノテーションの方法について、我々はどんなNMTシステムにも容易に適用できる三つの手法を示す  
NMTシステムをブラックボックスとして扱い、次に示す方法を用いてアノテートされたコーパスを用いて訓練する  

1. 教師なしalighnerを用いてアライメントを生成
1. アライメントの対応を使って、後処理ステップの単語翻訳で使用する単語辞書を構築
1. もし単語が辞書内にない場合はidentity translationを使用

### 3.1 Copyable Model
このアプローチでは, ソース文とターゲット文にある様々な未知語を表現するために, ただ1つのunkトークンでなく複数のトークンを導入する  
ソース文にあるOOV単語へ, 出現順にunk1,unk2.unk3と割り当て, 同じ未知語には同じトークンを割り当てる  
ターゲット文における未知語へのアノテーションはもうすこし複雑  
1. ソース文の未知語と対応が取られているターゲット文の各未知語は, ソース文と同じトークンで置き換えられる(ゆえに"Copyable"モデル)
1. アライメントがない, または既知単語と対応があるターゲット文の未知語には特殊なトークンunk∅が割り当てられる

### 3.2 Positional All Model(PosAll)
copyableモデルは,既知単語とアライメントが取られている単語は翻訳できないという制限があった(同一のトークンで置換されるため)  
これは, ソース語彙がターゲット語彙より大きくなりがちなためよく発生する  
この制限から, ソース文とターゲット文間の完全なアライメントを含むアノテーションを開発することを考えた  
(seq2seqでは)完全なアライメントが訓練時に利用可能であるため, 取得するのは容易(アテンションのこと言ってる？)  
このモデルではソース文では単一のunkトークンを用いる  
一方でターゲット文ではすべての単語の後ろにポジショントークン$p_d(d=-7,-6,-5,...-1,0,1,...,7)$を挿入する  
このポジショントークンは, その直前の単語の位置がjであるとき, ソース文の$i = j - d$番目と対応することを表す  
距離が離れすぎているアライメント(|d|>7?)はアライメントが取れていないとみなされ, それに限らずアライメントが取れない単語にはNULLトークン$p_n$が付けられる

### 3.3 Positional Unknown Model(PosUnk)
PosAllモデルの弱点はターゲット文の系列長が倍になってしまい, 学習が難しくなるしパラメータの更新が遅くなる  
でも, ワイらの後処理ステップが未知語のアライメントにのみ関係していることを踏まえると, 未知語にのみアノテートするほうがいいことは明白  
よって, $unkpos_d$というトークンを使って単語が未知であり, その未知語に対応するソース単語の相対的な位置dを表すPosUnkモデルを考えた  
PosAllモデルのように, アライメントがない未知語は$unkpos_∅$でおく  
またソース文のすべての未知語には同一のunkトークンを用いる  



## 4. Experiment
WMT14の英仏タスクで学習したOOVモデルの性能を評価  
翻訳の性能はnewstest2014の3003文をBLEUで計測  

### 4.1 Training Data
データにはWMT14の12M文を用いる  
ソフトマックス層の計算量を考慮して、仏(ターゲット)語彙を40k or 80kに、英(ソース)語彙を200kに頻度の高い順で設定した  
この語彙から漏れた単語は全部未知語として扱い、提案モデルで対処する対象とした  
アライメントの作成にはBerkeley alignerをデフォルト設定で使い、ソース/ターゲットどちらかが100単語を超える対訳は除いた  

### 4.2 Training Details
めんどくさい

### 4.3 A note on BLEU scores
デトークナイズしてうんしてホイする

### 4.4 Main Result
8LSTMのアンサンブル＋PosUnkモデルがSOTA